In [5]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import cv2

In [6]:
(train,test),info = tfds.load(
    'rock_paper_scissors',
    split=['train','test'],
    shuffle_files=True,
    as_supervised=True,
    batch_size=-1,
    with_info=True)

In [18]:
info

tfds.core.DatasetInfo(
    name='rock_paper_scissors',
    full_name='rock_paper_scissors/3.0.0',
    description="""
    Images of hands playing rock, paper, scissor game.
    """,
    homepage='http://laurencemoroney.com/rock-paper-scissors-dataset',
    data_path='C:\\Users\\justf\\tensorflow_datasets\\rock_paper_scissors\\3.0.0',
    file_format=tfrecord,
    download_size=219.53 MiB,
    dataset_size=219.23 MiB,
    features=FeaturesDict({
        'image': Image(shape=(300, 300, 3), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=3),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=372, num_shards=1>,
        'train': <SplitInfo num_examples=2520, num_shards=2>,
    },
    citation="""@ONLINE {rps,
    author = "Laurence Moroney",
    title = "Rock, Paper, Scissors Dataset",
    month = "feb",
    year = "2019",
    url = "http://laurencemoroney.com/rock-paper-scissors-dat

In [7]:
train_images,train_labels = tfds.as_numpy(train)

In [9]:
# count classes of train dataset 
np.unique(train_labels,return_counts=True)

(array([0, 1, 2], dtype=int64), array([840, 840, 840], dtype=int64))

In [10]:
test_images,test_labels = tfds.as_numpy(test)

In [11]:
# count classes of test dataset 
np.unique(test_labels,return_counts=True)

(array([0, 1, 2], dtype=int64), array([124, 124, 124], dtype=int64))

In [15]:
train_images[0].shape

(300, 300, 3)

In [10]:
def img_processing(img_data,labels):
    images = []
    for i in img_data:
        img = cv2.cvtColor(i,cv2.COLOR_BGR2RGB)
        img = cv2.resize(img,(224,224),interpolation=cv2.INTER_AREA)
        img = img.astype('float32')
        img /= 255
        images.append(img)

    labels = labels.astype('uint32')
    return np.array(images),labels

In [11]:
train_images,train_labels = img_processing(train_images,train_labels)
test_images,test_labels = img_processing(test_images,test_labels)

In [12]:
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(10000).batch(32)

test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(32)

# Tensorflow model from scratch

In [49]:
class ModelMine(tf.keras.Model):
    def __init__(self):
        super(ModelMine, self).__init__()

        self.conv1 = keras.layers.Conv2D(32,(3,3),activation='relu',padding = 'same')
        self.bn1   = keras.layers.BatchNormalization()

        self.conv2 = keras.layers.Conv2D(64,(3,3), activation='relu',padding = 'same')
        self.bn2   = keras.layers.BatchNormalization()

        self.conv3 = keras.layers.Conv2D(128,(3,3),activation = 'relu',padding = 'same')
        self.bn3   = keras.layers.BatchNormalization()

        
        self.max = keras.layers.MaxPooling2D((2,2))
        self.flatten = keras.layers.Flatten()

        
        self.fc1 = keras.layers.Dense(2048,activation='relu')
        self.drop1 = keras.layers.Dropout(0.25)

        self.fc2 = keras.layers.Dense(1024,activation='relu')
        self.drop2 = keras.layers.Dropout(0.2)

        self.fc3 = keras.layers.Dense(512,activation='relu')
        self.fc4 = keras.layers.Dense(100,activation='relu')
        self.fc5 = keras.layers.Dense(3,activation='softmax')



    def call(self, input_data):
        # conv layer 1
        x = self.conv1(input_data) # 32x224x224
        x = self.bn1(x) # 
        x = self.max(x) # 32x112x112

        # conv layer 2
        x = self.conv2(x) # 64x112x112
        x = self.bn2(x)
        x = self.max(x) # 64x56x56

        # conv layer 3
        x = self.conv3(x) # 128x56x56
        x = self.bn3(x)
        x = self.max(x) # 128x28x28

        #  dense layer 
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.drop1(x)
        x = self.fc2(x)
        x = self.drop2(x)
        x = self.fc3(x)
        x = self.fc4(x)
        return self.fc5(x)

In [50]:
model = ModelMine()

In [51]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

optimizer = tf.keras.optimizers.Adam()

In [52]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [53]:
@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images, training=True)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

In [54]:
@tf.function
def test_step(images, labels):
  predictions = model(images, training=False)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [55]:
EPOCHS = 20

for epoch in range(EPOCHS):
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

  for images, labels in train_ds:
    train_step(images, labels)

  for test_images, test_labels in test_ds:
    test_step(test_images, test_labels)

  print(
    f'Epoch {epoch + 1}, '
    f'Loss: {train_loss.result()}, '
    f'Accuracy: {train_accuracy.result() * 100}, '
    f'Test Loss: {test_loss.result()}, '
    f'Test Accuracy: {test_accuracy.result() * 100}'
  )

c:\Users\justf\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\backend.py:5585: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


Epoch 1, Loss: 6.989641189575195, Accuracy: 77.8968276977539, Test Loss: 2.65098237991333, Test Accuracy: 33.333335876464844
Epoch 2, Loss: 0.03307576850056648, Accuracy: 99.28571319580078, Test Loss: 9.465806007385254, Test Accuracy: 30.376344680786133
Epoch 3, Loss: 0.0974460244178772, Accuracy: 98.33333587646484, Test Loss: 10.371410369873047, Test Accuracy: 38.978492736816406
Epoch 4, Loss: 0.07244168221950531, Accuracy: 98.80952453613281, Test Loss: 8.093853950500488, Test Accuracy: 42.47311782836914
Epoch 5, Loss: 0.05818374082446098, Accuracy: 98.96825408935547, Test Loss: 10.961846351623535, Test Accuracy: 66.93548583984375


In [56]:
model.summary()

Model: "model_mine_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           multiple                  896       
                                                                 
 batch_normalization_9 (Batc  multiple                 128       
 hNormalization)                                                 
                                                                 
 conv2d_10 (Conv2D)          multiple                  18496     
                                                                 
 batch_normalization_10 (Bat  multiple                 256       
 chNormalization)                                                
                                                                 
 conv2d_11 (Conv2D)          multiple                  73856     
                                                                 
 batch_normalization_11 (Bat  multiple                

In [57]:
model.save('model1/')

INFO:tensorflow:Assets written to: model1/assets


INFO:tensorflow:Assets written to: model1/assets


# Model with transfer learning

In [21]:
densenet = keras.applications.DenseNet201(input_shape = (224,224,3),include_top=False)
densenet.trainable = False

In [19]:
densenet.summary()

Model: "densenet201"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 3)]                                                              
                                                                                                  
 zero_padding2d_2 (ZeroPadding2  (None, None, None,   0          ['input_2[0][0]']                
 D)                             3)                                                                
                                                                                                  
 conv1/conv (Conv2D)            (None, None, None,   9408        ['zero_padding2d_2[0][0]']       
                                64)                                                     

In [22]:
tr_model = Sequential([
    densenet,
    keras.layers.Flatten(),
    keras.layers.Dense(512,activation='relu'),
    keras.layers.Dense(100,activation='relu'),
    keras.layers.Dense(3,activation='softmax')
])

In [23]:
tr_model.compile(optimizer=keras.optimizers.Adam(),
                 loss=keras.losses.SparseCategoricalCrossentropy(),
                 metrics=keras.metrics.SparseCategoricalAccuracy()) 

In [24]:
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=2),
    tf.keras.callbacks.ModelCheckpoint(filepath='saved_model/model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
]
tr_model.fit(
    train_ds,
    epochs=5,
    callbacks=my_callbacks,
    verbose = 1,
    validation_data = test_ds)

Epoch 1/5
79/79 [==============================] - 327s 4s/step - loss: 1.5368 - sparse_categorical_accuracy: 0.9286 - val_loss: 1.9792 - val_sparse_categorical_accuracy: 0.8441
Epoch 2/5
79/79 [==============================] - 307s 4s/step - loss: 3.2793e-07 - sparse_categorical_accuracy: 1.0000 - val_loss: 0.9833 - val_sparse_categorical_accuracy: 0.8710
Epoch 3/5
79/79 [==============================] - 8768s 112s/step - loss: 8.9407e-09 - sparse_categorical_accuracy: 1.0000 - val_loss: 1.0010 - val_sparse_categorical_accuracy: 0.8737
Epoch 4/5
79/79 [==============================] - 363s 5s/step - loss: 6.1970e-09 - sparse_categorical_accuracy: 1.0000 - val_loss: 1.0161 - val_sparse_categorical_accuracy: 0.8763


In [26]:
tr_model.evaluate(test_ds)

12/12 [==============================] - 26s 2s/step - loss: 1.0161 - sparse_categorical_accuracy: 0.8763


[1.016093373298645, 0.8763440847396851]